<a href="https://colab.research.google.com/github/Tada-Disney/Portfolio-/blob/main/A_B_Testing_on_Globox_Website_Banner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing the necessary libaries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# loading the globox datasets
globox_data = pd.read_csv('/Globox_dataset.csv')

In [ ]:
# view on datasets contents
globox_data.head()

,id,country,gender,device,group,conversion,total_spent
0,1000000,CAN,M,I,B,No,0.0
1,1000001,BRA,M,A,A,No,0.0
2,1000002,FRA,M,A,A,No,0.0
3,1000003,BRA,M,I,B,No,0.0
4,1000004,DEU,F,A,A,No,0.0


In [ ]:
globox_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48943 entries, 0 to 48942
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           48943 non-null  int64  
 1   country      48943 non-null  object 
 2   gender       48943 non-null  object 
 3   device       48943 non-null  object 
 4   group        48943 non-null  object 
 5   conversion   48943 non-null  object 
 6   total_spent  48943 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 2.6+ MB


In [ ]:
globox_data.describe()

,id,total_spent
count,4.894300e+04,48943.000000
mean,1.025001e+06,3.382735
std,1.443521e+04,25.674940
min,1.000000e+06,0.000000
25%,1.012506e+06,0.000000
50%,1.024996e+06,0.000000
75%,1.037504e+06,0.000000
max,1.049999e+06,1659.400000


Conduct a hypothesis test to see whether there is a difference in the conversion rate between the two groups. What are the resulting p-value and conclusion?

H0: p1 = p2,

Ha: p1 != p2

In [ ]:
from statsmodels.stats.proportion import proportions_ztest
import scipy.stats as st
from scipy.stats import t

In [ ]:
# significance level
alpha = 0.05

In [ ]:
# value count
Value_count = globox_data.groupby('group')['conversion'].value_counts()

In [ ]:
print(Value_count)

group  conversion
A      No            23388
       Yes             955
B      No            23461
       Yes            1139
Name: conversion, dtype: int64


In [ ]:
# Success count
success_counts = np.array([955, 1139])

In [ ]:
# number of rows
n= np.array([23388+955, 23461+1139])

In [ ]:
# Getting our pValue from proportion_ztest
stat, p_value = proportions_ztest(count=success_counts, nobs=n,
                                  alternative="two-sided")

In [ ]:
# printint the pvalue
print(stat, p_value)

-3.864291770414927 0.00011141198532947085


In [ ]:
print(p_value <= alpha)

True


The P-value is less than our significance level so
we have enough evidence to reject our null hypothesis. There fore, we would conclude there is statiscally significant difference in the number of conversion between treatment and control group.

What is the 95% confidence interval for the difference in the conversion rate between the treatment and control (treatment-control)?

In [ ]:
# calculating sample size, number of success for group A
group_a = globox_data[globox_data['group']=='A']
n_A = len(group_a)
x_A = 955

In [ ]:
# calculating sample size, number of success for group B
group_b = globox_data[globox_data['group']=='B']
n_B = len(group_b)
x_B = 1139

In [ ]:
# calculate the sample size proportion
p1 = x_A/ n_A
p2 = x_B/ n_B

In [ ]:
# confidence level
ci = 0.95

In [ ]:
# Find the z-critical value
z = st.norm.ppf((1 + ci) / 2) # Use the inverse cumulative distribution function

In [ ]:
# Calculate the standard error of the difference
se = ((p1 * (1 - p1) / n_A) + (p2 * (1 - p2) / n_B)) ** 0.5

In [ ]:
# Calculate the margin of error
me = z * se

In [ ]:
# Calculate the lower and upper bounds of the confidence interval
lower = p2 - p1 - me
upper = p2 - p1 + me
print(lower, upper)

0.0034860511629807105 0.0106535939963596


Conduct a hypothesis test to see whether there is a difference in the average amount spent per user between the two groups. What are the resulting p-value and conclusion?


H0: μA - μB = 0

H1: μA - μB != 0


In [ ]:
# Subset the data for group A and calculate the mean, standard deviation, and sample size
group_a = globox_data[globox_data['group']=='A']
x_a = group_a['total_spent'].mean()
s_a = group_a['total_spent'].std()
n_a = len(group_a)

# Subset the data for group B and calculate the mean, standard deviation, and sample size
group_b = globox_data[globox_data['group']=='B']
x_b = group_b['total_spent'].mean()
s_b = group_b['total_spent'].std()
n_b = len(group_b)

# Calculate the numerator and denominator for the test statistic
numerator = x_b - x_a
denominator = np.sqrt(s_b ** 2 / n_b + s_a ** 2 / n_a)

# Calculate the test statistic and the degrees of freedom
t_stat = numerator / denominator
degrees_of_freedom = n_a + n_b - 2

# Calculate the two-sided p-value using the survival function
p_value = 2 * t.sf(abs(t_stat), df=degrees_of_freedom)

# Print the results
print(f"The test statistic is {t_stat:.2f} with {degrees_of_freedom} degrees of freedom.")
print(f"The two-sided p-value is {p_value:.4f}.")

The test statistic is 0.07 with 48941 degrees of freedom.
The two-sided p-value is 0.9439.


since the resulting p_value is greater than the significance level of
5%, we do not have enough evidence to reject the null hypthesis. there fore, we would conclude that there is no statiscally significant difference in the average amount spent per user between the two group.

What is the 95% confidence interval for the difference in the average amount spent per user between the treatment and the control (treatment-control)?

In [ ]:
# Control group
x_A = globox_data.groupby('group')['total_spent'].mean()['A']
s_A = globox_data.groupby('group')['total_spent'].std()['A']
n_A = globox_data.groupby('group')['total_spent'].count()['A']

In [ ]:
# Treatment group
x_B = globox_data.groupby('group')['total_spent'].mean()['B']
s_B = globox_data.groupby('group')['total_spent'].std()['B']
n_B = globox_data.groupby('group')['total_spent'].count()['B']

In [ ]:
# Calculate the degree of freedom
nu = ((s_B**2/n_B + s_A**2/n_A)**2) / ((s_B**4/(n_B**2*(n_B-1))) + (s_A**4/(n_A**2*(n_A-1))))

In [ ]:
# Calculate the critical value
alpha = 0.05
t_crit = t.ppf(1 - alpha/2, nu)

In [ ]:

# Calculate the standard error of the difference
se_diff = np.sqrt(s_B**2/n_B + s_A**2/n_A)

In [ ]:
# Calculate the confidence interval
ci_low = (x_B - x_A) - t_crit * se_diff
ci_high = (x_B - x_A) + t_crit * se_diff

In [ ]:
# Print the result
print(f"The 95% confidence interval for the difference in the average amount spent per user between the treatment and control groups is ({ci_low:.2f}, {ci_high:.2f}).")

The 95% confidence interval for the difference in the average amount spent per user between the treatment and control groups is (-0.44, 0.47).
